# Penny Stock Model

In [15]:
# import libraries
import pandas as pd
import yfinance as yf
import numpy as np
import datetime

In [16]:
# parameters
VolRise = 0.99  # open position, after current volume will be more than VolRise value
ExitAfter = 0.25 # close position after price achieve ExitAfter
StopLoss = -0.07 # close position after price meets stop
RiskPertrade = 2000 # buy for sum 
TradeAfterDays = 3 # begin trade after TradeAfterDays days from start
price_filter = 0.5 # below this price we open pos
AvgVolDayLimit = 10 # average for shifting average 




In [17]:
def good_price(ticker_name,date_from_list):
    
    # function to calculate appropriate price
    res = False
    stock_ticker = yf.Ticker(ticker_name)
    r = stock_ticker.history(max,"1d",date_from_list)
    price = r.iloc[0]["Close"]
    if price<price_filter:
        res = True

    return res


In [18]:
def OpenPosSig():
    pass

In [19]:
#reed.history("1mo","1d","2022-10-23","2023-01-10")


In [20]:
# load ticker list from excel-file
list_stocks = pd.read_excel('inputFiles/list_stocks.xlsx', index_col ='Company')
all_res = []
data_group=[]
data_group_res=[]

stock_res =[]
stock_small_res=[]
#len(list_stocks)

for i in range(len(list_stocks)): # потім прописати весь список
    symbol = list_stocks.index[i] # беремо акцію по індексу

    datefromlist = list_stocks.iloc[i]['NotDate']
    #print('symbol=',symbol)
    #print('datefromlist=',datefromlist)
    if good_price(symbol,datefromlist)==False: # if price meet conditions
        continue

    # to calc average volume use 20 days before
    start_date = datefromlist - datetime.timedelta(days=19)                                           
    end_date = datefromlist + datetime.timedelta(days=180)

    

    #load stock price
    #symbol = list_stocks.index[i] # беремо акцію по індексу
    wdf = yf.download(symbol, start_date, end_date)



    # building features 
    wdf['avgvol']= wdf['Volume'].rolling(window=AvgVolDayLimit, win_type=None).mean()  # середній обєм за 10 днів
    wdf['VolRise'] =wdf['Volume']/wdf['avgvol']-1      # building vol signal
    wdf['volsignal']=wdf['VolRise']>VolRise            # volume signal column
    wdf['GreenCandleSig'] = wdf['High']>wdf['Close']   # green candle
    wdf['TradeAfter']=wdf.index

    wdf['TradeAfterDaysSig'] = wdf['TradeAfter'] > datefromlist + datetime.timedelta(days=TradeAfterDays)
    wdf['OpenPrice']=0
    wdf['Profit']=0
    wdf['StopPrice']=0
    wdf['TargetPrice']=0
    wdf['Ticker']=symbol
    del(wdf['TradeAfter'])
    lastrow=wdf.tail(1).index # last row in order to stop calculations

    # main module
    PriceOpen = 0
    StopPrice = 0
    TPPrice = 0
    CloseSignal = 0
    status = 0
    
    
    for index, row in wdf.iterrows():  # change to itter....
        # looking for opening signal 
        if (PriceOpen ==0) & (lastrow[0]!=index): # if we haven't opened position and it's not last row
            
            if row['volsignal'] & row['TradeAfterDaysSig'] & row['GreenCandleSig']:  # if Volume > than AvgVolume and green candle
                if row["Close"]>price_filter: # if price for opening is bigger than limit - please skip it
                    continue

                PriceOpen = row['Close']  # record close price as price of open
                wdf.at[index,'OpenPrice']= PriceOpen
                StopPrice = PriceOpen-(PriceOpen*StopLoss*-1) # setting stop-loss

                TPPrice = PriceOpen+(PriceOpen*ExitAfter)  # set target price
                status=1

    #     # working with first opened position
        if (PriceOpen >0) & (status==0):
            wdf.at[index,'OpenPrice']= PriceOpen
            wdf.at[index,'StopPrice']= StopPrice
            wdf.at[index,'TargetPrice']= TPPrice


            if row['High']>TPPrice:
                wdf.at[index,'Profit']= RiskPertrade*ExitAfter
                PriceOpen =0
                TPPrice=0
                StopPrice=0

            if row['Low']<StopPrice:
                wdf.at[index,'Profit']= RiskPertrade*StopLoss
                PriceOpen =0
                TPPrice=0
                StopPrice=0


        status = 0    

        # if there is open position and its last row
        if lastrow[0]==index:
            if PriceOpen >0:
                wdf.at[index,'Profit']=(row['Close']/PriceOpen-1)*RiskPertrade
                PriceOpen =0
                TPPrice=0
                StopPrice=0
   
    
    #тут закінчується блок перебору строк акції і йде на наступну акцію
    # 1. тікер
    # 2. profit
    # 3. tradecount
    # 4. ProfitTrades
    #  5.   LossTrades
    # 6.    WinRate
    #7.     AvgHold'
    
    tradecount=(wdf['Profit']!=0).sum()
    ProfitTrades = (wdf['Profit']>0).sum()
    LossTrades = (wdf['Profit']<0).sum()
    #WinRate = round(ProfitTrades/tradecount,1)
    #AvgHold = round((wdf['OpenPrice']!=0).sum()/tradecount,1)
    stock_small_res=[symbol,round(wdf['Profit'].sum(),2),tradecount,ProfitTrades,LossTrades]
    
    data_group=wdf['Profit']
    stock_res.append(stock_small_res)
    data_group_res.append(data_group)
    # блок запису dataFrame:
    all_res.append(wdf)
      
    df_output = pd.concat(all_res)
    
    df_data =pd.concat(data_group_res)
    #df_data.columns['Profit']
   # mask1 = df_data.Profit >0
    #mask2 = df_data.Profit <0
    #df_out= df_data.loc[df_data['Profit']!=0]



    
      
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [21]:
df_output

,Open,High,Low,Close,Adj Close,Volume,avgvol,VolRise,volsignal,GreenCandleSig,TradeAfterDaysSig,OpenPrice,Profit,StopPrice,TargetPrice,Ticker
Date,,,,,,,,,,,,,,,,
2022-01-24,0.163029,0.165659,0.152511,0.165659,-0.184924,2872406,NaN,NaN,False,False,False,0.000,0.0,0.00000,0.0000,CBIO
2022-01-25,0.178806,0.197213,0.152511,0.155141,-0.173183,24439979,NaN,NaN,False,True,False,0.000,0.0,0.00000,0.0000,CBIO
2022-01-26,0.152511,0.160400,0.147252,0.149882,-0.167312,4451411,NaN,NaN,False,True,False,0.000,0.0,0.00000,0.0000,CBIO
2022-01-27,0.152511,0.157770,0.139364,0.144623,-0.161441,3194900,NaN,NaN,False,True,False,0.000,0.0,0.00000,0.0000,CBIO
2022-01-28,0.144623,0.152511,0.139364,0.144623,-0.161441,1959306,NaN,NaN,False,True,False,0.000,0.0,0.00000,0.0000,CBIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-11,0.170000,0.173000,0.166000,0.172000,0.172000,4068300,6819910.0,-0.403467,False,True,True,0.000,0.0,0.00000,0.0000,RDHL
2023-01-12,0.170000,0.180000,0.168000,0.179000,0.179000,3875100,6997920.0,-0.446250,False,True,True,0.000,0.0,0.00000,0.0000,RDHL
2023-01-13,0.200000,0.206000,0.182000,0.198000,0.198000,26571800,9415340.0,1.822182,True,True,True,0.198,0.0,0.00000,0.0000,RDHL


In [22]:
stock_df = pd.DataFrame(stock_res, columns = ['Stock','Profit','TradeCount','ProfitTrades','LossTrades'])
df_day = pd.DataFrame(df_data )
df_day.index.name ="Date"

In [23]:
df_day = df_day.loc[df_day['Profit']!=0] # remove 0 values
df_day = df_day.groupby([df_day.index.date]).sum() # групуємо по даті


In [24]:
df_day.sort_index()


,Profit
2022-03-03,500.000000
2022-03-17,-140.000000
2022-03-25,500.000000
2022-03-31,-140.000000
2022-04-12,-140.000000
...,...
2023-01-10,-280.000000
2023-01-12,-60.000000
2023-01-13,220.000000
2023-01-17,360.000000


In [25]:
with pd.ExcelWriter("outputFiles/output.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_output.to_excel(writer, sheet_name="full_B")
    df_day.to_excel(writer, sheet_name="by date")
    stock_df.to_excel(writer, sheet_name="statByStock")

In [26]:
#testDF = yf.download("AEI", "2022-10-01", "2022-12-10",back_adjust=True)